In [174]:
import pandas as pd
import numpy as np

In [175]:
messages_df = pd.concat(
    (pd.read_csv('.\csv\chat_messages_1.csv',low_memory=False),
     pd.read_csv('.\csv\chat_messages_2.csv',low_memory=False))
    ).dropna()
messages_df.head(5)
messages_df.columns

Index(['account_id', 'alliance_id', 'timestamp', 'date', 'raw_message',
       'filtered_message', 'filtered', 'filtered_content', 'risk',
       'filter_detected_language', 'is_family_friendly', 'GENERAL_RISK',
       'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY',
       'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS',
       'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE',
       'SENTIMENT', 'POLITICS'],
      dtype='object')

In [176]:
messages_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362766 entries, 0 to 182827
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   account_id                   362766 non-null  object 
 1   alliance_id                  362766 non-null  object 
 2   timestamp                    362766 non-null  object 
 3   date                         362766 non-null  object 
 4   raw_message                  362766 non-null  object 
 5   filtered_message             362766 non-null  object 
 6   filtered                     362766 non-null  object 
 7   filtered_content             362766 non-null  object 
 8   risk                         362766 non-null  object 
 9   filter_detected_language     362766 non-null  object 
 10  is_family_friendly           362766 non-null  object 
 11  GENERAL_RISK                 362766 non-null  object 
 12  BULLYING                     362766 non-null  object 
 13 

## Find triggered messages

In [177]:
dictionnary_account = np.array(["snap", "snapchat", "insta", "instagram", "skype", "omegle", "zoom", "@?", "@ ?", "whatsapp", "adress", "phone number", "phonenumber", "vk", "discord", 'social','media'])
dictionnary_compliments = np.array(["cutie", "horny", "sweetheart", "sweety"])
dictionnary_relationship = np.array(["boyfriend", "girlfriend", "relationship","gf","bf"])
dictionnary_smileyHArt = np.array(["\u2764\uFE0F","love"])

dictionnary_trigger = np.concatenate([
	dictionnary_account,
	dictionnary_compliments,
	dictionnary_relationship
])

def is_risky(message):
	
	words = np.array(message.split())
	for word in words:
		if word.lower() in dictionnary_trigger:
			return 1
	return 0

In [178]:
messages_df['trigger'] = messages_df['raw_message'].apply(is_risky)
messages_df['trigger']


0         0
1         0
2         0
3         0
4         0
         ..
182823    0
182824    0
182825    0
182826    0
182827    0
Name: trigger, Length: 362766, dtype: int64

In [179]:
messages_df['trigger'].sum()

351

In [180]:
messages_df['raw_message'][[10247]]

10247            where dumeny @?
10247    for the first time ever
Name: raw_message, dtype: object

## Get alliance conversation for every trigger
create a df per alliance and sort each df by date

In [181]:
alliance_dataframe = pd.read_csv('.\csv\\alliance_membership.csv',low_memory=False)

In [182]:
alliance_ids_triggered = messages_df.loc[messages_df['trigger']==1]["alliance_id"].drop_duplicates()
len(alliance_ids_triggered)

253

In [183]:
triggered_alliances_df = [messages_df.loc[messages_df['alliance_id']==alliance].sort_values(by=["timestamp"], ignore_index=True) for alliance in alliance_ids_triggered]

temp = triggered_alliances_df[2]
print(temp.head())
temp.loc[temp['trigger']==1]

                                          account_id  \
0  0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...   
1  d00ea22e43dd5647d6854e27b6b319cd5b1180f2cbb714...   
2  0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...   
3  d00ea22e43dd5647d6854e27b6b319cd5b1180f2cbb714...   
4  d00ea22e43dd5647d6854e27b6b319cd5b1180f2cbb714...   

                                         alliance_id             timestamp  \
0  6f083e8432eab1c091d4d083e0acc2942947cdb5639740...  20230301T021259.079Z   
1  6f083e8432eab1c091d4d083e0acc2942947cdb5639740...  20230301T022519.126Z   
2  6f083e8432eab1c091d4d083e0acc2942947cdb5639740...  20230301T022537.643Z   
3  6f083e8432eab1c091d4d083e0acc2942947cdb5639740...  20230301T022550.317Z   
4  6f083e8432eab1c091d4d083e0acc2942947cdb5639740...  20230301T022613.201Z   

         date                 raw_message            filtered_message  \
0  2023-03-01                         hey                         hey   
1  2023-03-01                        heyy       

,account_id,alliance_id,timestamp,date,raw_message,filtered_message,filtered,filtered_content,risk,filter_detected_language,...,HATE_SPEECH,RELIGIOUS,WEBSITE,CHILD_GROOMING,PUBLIC_THREAT,EXTREMISM,SUBVERSIVE,SENTIMENT,POLITICS,trigger
43,0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...,6f083e8432eab1c091d4d083e0acc2942947cdb5639740...,20230301T070956.276Z,2023-03-01,need a bf,need a **,1.0,['bf'],3.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
80,0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...,6f083e8432eab1c091d4d083e0acc2942947cdb5639740...,20230301T082916.202Z,2023-03-01,my boyfriend was also like you guys. very toxic,my ********* was also like you guys. very toxic,1,['boyfriend'],3.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
123,0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...,6f083e8432eab1c091d4d083e0acc2942947cdb5639740...,20230301T084518.197Z,2023-03-01,my boyfriend gave me money whenever we go at h...,my ********* gave me money whenever we go at h...,1.0,['boyfriend'],3.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
444,0e38df97bdc7187a5c66873a65f44c2d6c6562976db98b...,6f083e8432eab1c091d4d083e0acc2942947cdb5639740...,20230301T164318.881Z,2023-03-01,i need bf for pass,i need ** for pass,1,['bf'],3,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [184]:
def print_messages_of_Dataframe(df):
    for msg in df['raw_message']:
        print(msg)
print_messages_of_Dataframe(temp)

hey
heyy
goodmorning
goodmorning 😀
normally I woke up at 1 pm
why
because I sleep late
about 4 5 am
oh
why
so you didn't sleep till now?
i also don't know
yups
n u?
Same
sorry I can't share that
no worries
yeah i guess so
yups 👍
hello
guys
hooiii
bhadiya
I want the ram rider but no trade token 😫😩
😭😭sad
Yea 😩
hua
Hello people 🫡
hello hello everyone
ok
hello
if we have 4 active
I am in 🤓
yash
ok
well played
thanks
Revenge OP 😝😝😝😝
draft is not my thing
yupz
normal 2v2
normal one
nobody can beat me
need a bf
my @bf cheated on me
so?
what made you think it is a relevant topic to discuss in this game?
please have some sense, or if you don't, just leave the clan.
some one asked me
that's why i said
then why did you ask a fellow player to buy you a pass?
because I'm a student
many PPL here have passes
why did you expect someone would buy you a pass?
I think he is fake id
i know.
be aware guys don't waste money
if i was a Co leader, i would kick this one out of the clan.
What did i do
FAKE ID H

In [185]:
alliance_id_list = pd.read_csv('.\csv\\alliance_membership.csv',low_memory=False).drop_duplicates("alliance_id").dropna()

private_chats = alliance_id_list.loc[alliance_id_list["size"]==2]["alliance_id"]
private_chats_messages = messages_df.loc[messages_df["alliance_id"].isin(private_chats)]
private_chats_messages['trigger'] = private_chats_messages['raw_message'].apply(is_risky)
private_chats_ids_triggered = private_chats_messages.loc[private_chats_messages['trigger']==1]["alliance_id"].drop_duplicates()
triggered_private_chats_df = [messages_df.loc[messages_df['alliance_id']==alliance].sort_values(by=["timestamp"], ignore_index=True) for alliance in alliance_ids_triggered]
temp = triggered_alliances_df[1]
temp.loc[temp['trigger']==1]

C:\Users\Andru\AppData\Local\Temp\ipykernel_13256\564372644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  private_chats_messages['trigger'] = private_chats_messages['raw_message'].apply(is_risky)


,account_id,alliance_id,timestamp,date,raw_message,filtered_message,filtered,filtered_content,risk,filter_detected_language,...,HATE_SPEECH,RELIGIOUS,WEBSITE,CHILD_GROOMING,PUBLIC_THREAT,EXTREMISM,SUBVERSIVE,SENTIMENT,POLITICS,trigger
123,066b0c39b0d4faf8fa78ff9cfe291e39d314f614983ce9...,40c3b750f63c5c87db4653f17d3e2f6b10636222f7d196...,20230301T193715.580Z,2023-03-01,do you gf or something 😏,do you gf or something 😏,0,[],3.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
179,066b0c39b0d4faf8fa78ff9cfe291e39d314f614983ce9...,40c3b750f63c5c87db4653f17d3e2f6b10636222f7d196...,20230301T194858.608Z,2023-03-01,it's really hard get gf in our country,it's really hard get gf in our country,0,[],3,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
190,066b0c39b0d4faf8fa78ff9cfe291e39d314f614983ce9...,40c3b750f63c5c87db4653f17d3e2f6b10636222f7d196...,20230301T195104.333Z,2023-03-01,and our country society gossip when youg kids ...,and our country society gossip when youg kids ...,0,[],3,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
237,2c2a73e82bcbb83e77edb43fa064188eac86b2b3765912...,40c3b750f63c5c87db4653f17d3e2f6b10636222f7d196...,20230301T200435.120Z,2023-03-01,Also same height as my gf lmfao,Also same height as my gf lmfao,0,[],3,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


# Token charge model
## Build suspect features for model 

2 Token charge model - harder check because less data

	Each elements will add a certain 

	Elements fixes courts termes et long terme

		Fixe:

			- give account 

			- Ask age DATA

			- Young age with old age

		Short term:

			- Asks a lot a messages in a row before an answer DATA

			- use the Risk_values DATA #TODO ADDED THIS

		Long term: 

			- change often alliances DATA

			- Send same first message everytime DATA

			- temps avant de se faire report depuis qu'il join une alliance DATA

In [186]:
#GENERAL FUCNTIONS

def get_alliance_conv(alliance_id):
    if not (alliance_id in messages_df['alliance_id'].values):
        return None
    alliance_df = messages_df[messages_df['alliance_id'] == alliance_id]
    return alliance_df


def get_alliance_conv_triggered(alliance_id):
    for df in triggered_alliances_df:
        if (df.iloc[0]['alliance_id'] == alliance_id):
            return df 

alliance_id = triggered_alliances_df[0].iloc[0]['alliance_id']
print("alliance_id:",alliance_id,"type:",type(alliance_id))
df = get_alliance_conv_triggered(alliance_id)
df.head()

alliance_id: 5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef584c3bf71ae5916cf3c type: <class 'str'>


,account_id,alliance_id,timestamp,date,raw_message,filtered_message,filtered,filtered_content,risk,filter_detected_language,...,HATE_SPEECH,RELIGIOUS,WEBSITE,CHILD_GROOMING,PUBLIC_THREAT,EXTREMISM,SUBVERSIVE,SENTIMENT,POLITICS,trigger
0,aabb7cc4eb798f476925f8130fb5d29f431640f819ceee...,5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef5...,20230301T031505.870Z,2023-03-01,goodu morning,goodu morning,0,[],1,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,aabb7cc4eb798f476925f8130fb5d29f431640f819ceee...,5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef5...,20230301T035731.560Z,2023-03-01,ohh ye mausam bhi kharab horakha itna,ohh ye mausam bhi ****** ******* itna,1,['kharab' 'horakha'],5,en,...,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
2,aabb7cc4eb798f476925f8130fb5d29f431640f819ceee...,5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef5...,20230301T035738.449Z,2023-03-01,khatab,khatab,0,[],2.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
3,aabb7cc4eb798f476925f8130fb5d29f431640f819ceee...,5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef5...,20230301T035747.719Z,2023-03-01,horakhaa,horakhaa,0,[],5.0,en,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,aabb7cc4eb798f476925f8130fb5d29f431640f819ceee...,5760ff059cf3105fa9ea1c7b4eae26ca7c773dc9f6eef5...,20230301T051519.194Z,2023-03-01,par video private hai link bhejungi tabhi dikhegi,par video private hai link bhejungi tabhi dikhegi,0.0,[],5.0,en,...,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [204]:
# FEATURES ON MESSAGES

def is_msg_suspect_with_dictionnary(message, dic_to_check):
	dictionnary = dic_to_check
	words = np.array(message.split())
	for word in words:
		if word.lower() in dictionnary:
			return 1
	return 0


def check_message_ask_age(msg):
    # go with the dictionnary value through the message and check if the message the value matches one sequence of the message
    # in the dictionary give me different to ask for age
	
    dic = {"age":
["I am %number", "How old"
"What age",
"Wat age"
"tell me ur age?",
"ur age",
"your agee",
"u young",
"u 10",
"u 11",
"u 12",
"born",
"young",
"same age",
"im %number"
]}  
    #for word in msg.split():
     #   if word.lower() == dic["age"]:
      #      return is_msg_suspect_with_dictionnary(msg,dic)

    return is_msg_suspect_with_dictionnary(msg,dic)

def check_message_trigger(msg):
	return is_msg_suspect_with_dictionnary(msg,dictionnary_trigger)

def check_message_account(msg):
	return is_msg_suspect_with_dictionnary(msg,dictionnary_account)

def check_message_compliment(msg):
	return is_msg_suspect_with_dictionnary(msg,dictionnary_compliments)

def check_message_relationship(msg):
	return is_msg_suspect_with_dictionnary(msg,dictionnary_relationship)

def count_triggers_in_alliance_conv(alliance_id):
	alliance_conv = get_alliance_conv(alliance_id)
	return alliance_conv['trigger'].sum()

def count_triggers_account(account):
	messages_account = messages_df[messages_df['account_id']==account]
	return messages_account['trigger'].sum()


allinace_id_with_trigger = triggered_alliances_df[0].iloc[0]['alliance_id']
print('count_triggers_in_alliance_conv(allinace_id_with_trigger)',count_triggers_in_alliance_conv(allinace_id_with_trigger))

account = '085351ff2d1bb87dabcf8cc7af14383e271382dc325fb31841cb2a75c3276482'
print('account', account)
print('count_triggers_account(account)', count_triggers_account(account))


count_triggers_in_alliance_conv(allinace_id_with_trigger) 2
account 085351ff2d1bb87dabcf8cc7af14383e271382dc325fb31841cb2a75c3276482
count_triggers_account(account) 0


In [188]:
#FUNCTIONS ON ACCOUNTS

from datetime import timedelta

TOTAL_TIME_DATA = timedelta(
    days=1,
    seconds=0,
    microseconds=0,
    milliseconds=0,
    minutes=0,
    hours=0,
    weeks=0
)

often_time_data = timedelta(
    days=0,
    seconds=0,
    microseconds=0,
    milliseconds=0,
    minutes=0,
    hours=12,
    weeks=0
)

def nb_of_alliance_changes(account):
    his_alliances = alliance_dataframe[alliance_dataframe['account_id'] == account]['alliance_id'].drop_duplicates()
    print("his_alliances",his_alliances)
    return len(his_alliances)

account = alliance_dataframe.iloc[[602]]['account_id'].values[0]
print("account: ",account)

print("nb_of_alliance_changes(account):",nb_of_alliance_changes(account))

def often_change_alliance(account,often_timedata):
    his_alliances = alliance_dataframe[alliance_dataframe['account_id'] == account]
    nb_all = nb_of_alliance_changes(account)
    if nb_all==1:
        return 0
    else:
        nb_time_per_total = TOTAL_TIME_DATA.total_seconds() / often_timedata.total_seconds()
        print('nb_time_per_total:',nb_time_per_total)
        nb_changes_per_day = int(np.ceil(nb_time_per_total))
        print('nb_changes_per_day:',nb_changes_per_day)
        return nb_all> nb_changes_per_day


print('often_change_alliance(account,often_time_data):',often_change_alliance(account,often_time_data))

account:  085351ff2d1bb87dabcf8cc7af14383e271382dc325fb31841cb2a75c3276482
his_alliances 583    35ac928cec57517911e748cae14f3e3b93042786bd3e03...
602    dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
Name: alliance_id, dtype: object
nb_of_alliance_changes(account): 2
his_alliances 583    35ac928cec57517911e748cae14f3e3b93042786bd3e03...
602    dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
Name: alliance_id, dtype: object
nb_time_per_total: 2.0
nb_changes_per_day: 2
often_change_alliance(account,often_time_data): False


In [189]:
#FUNCTIONS ON ACCOUNTS 2

def same_message_alliance(account):
    his_alliances_id = alliance_dataframe[alliance_dataframe['account_id'] == account]['alliance_id'].drop_duplicates().values
    print('his_alliances_id',his_alliances_id)
    nb_all = nb_of_alliance_changes(account)
    print("nb_all:",nb_all)
    if nb_all==1:
        return 0
    else:
        firstmsg_alliance = []
        for alliance_id in his_alliances_id:
            alliance_conv_df = get_alliance_conv(alliance_id)
            print('alliance_conv_df len:',len(alliance_conv_df))
            if alliance_conv_df is None: #no messages in that alliance
                continue
            if not(account in alliance_conv_df['account_id'].values): #account did not send any message in that alliance
                continue 
            msgs_sent_in_conv = alliance_conv_df[alliance_conv_df['account_id']==account]['raw_message'].reset_index(drop=True)
      
            #print('msgs_sent_in_conv',msgs_sent_in_conv)
            firstmsg_alliance.append(msgs_sent_in_conv[0])
            print('firstmsg_alliance',firstmsg_alliance)

        return not (len(firstmsg_alliance) == len(set(firstmsg_alliance)) )
    
    
print("account: ",account)
return_value = same_message_alliance(account)
print('same_message_alliance(account):',return_value)

alliances_id = ['35ac928cec57517911e748cae14f3e3b93042786bd3e03f23e9ae2dccd73fa31'
 'dbd78552c212677fa5e1414e8ec32ee40beabd16e958533340c9e3a99f6a99d8']

alliances_in_which_account_send_message = messages_df[messages_df['account_id']==account]['alliance_id']
print('alliances_in_which_account_send_message',alliances_in_which_account_send_message)
print('alliances_in_which_account_send_message uniques',alliances_in_which_account_send_message.drop_duplicates())
print('35ac928cec57517911e748cae14f3e3b93042786bd3e03f23e9ae2dccd73fa31' in messages_df['alliance_id'].values)


account:  085351ff2d1bb87dabcf8cc7af14383e271382dc325fb31841cb2a75c3276482
his_alliances_id ['35ac928cec57517911e748cae14f3e3b93042786bd3e03f23e9ae2dccd73fa31'
 'dbd78552c212677fa5e1414e8ec32ee40beabd16e958533340c9e3a99f6a99d8']
his_alliances 583    35ac928cec57517911e748cae14f3e3b93042786bd3e03...
602    dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
Name: alliance_id, dtype: object
nb_all: 2
alliance_conv_df len: 104
alliance_conv_df len: 50
firstmsg_alliance ['reckon u could do us the biggest favor?']
same_message_alliance(account): False
alliances_in_which_account_send_message 8570      dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
40091     dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
50009     dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
54063     dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
79957     dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
131658    dbd78552c212677fa5e1414e8ec32ee40beabd16e95853...
149517    dbd78552c212677fa5e1414e8ec32ee40beabd1

In [194]:
#FUNCTION ON ACCOUNTS 3

def timestamp_to_number(timestamp):
    timestamp = timestamp[9:-1]
    hours = int(timestamp[:2])
    minutes = int(timestamp[2:4])
    seconds = float(timestamp[4:])
    return hours*60 + minutes +seconds/60

def check_time_kick(account):
    alliances = alliance_dataframe  
    logs = alliances.loc[alliances['account_id']==account].reset_index()
    timestamp_diff = []
    if 'kicked_out' in logs['action'].unique():
        id_kicks = logs.loc[logs['action']=='kicked_out'].index
        for id_kick in id_kicks:
            if id_kick !=0:
                timestamp_diff.append(timestamp_to_number(logs['timestamp'].iloc[id_kick]) - 
                                    timestamp_to_number(logs['timestamp'].iloc[id_kick-1]))
    if timestamp_diff==[]:
        return 0
    return 1-np.mean(timestamp_diff)/(24*60)

def number_kicks(account):
    alliances = alliance_dataframe  
    logs = alliances.loc[alliances['account_id']==account].reset_index()
    kicks = logs.loc[logs['action']=='kicked_out']
    return kicks.shape[0]

def small_clan(account):
    alliances = alliance_dataframe  
    alliances = alliances.drop_duplicates("alliance_id").dropna()
    private_chats = alliance_dataframe.loc[alliance_dataframe["size"]<=4]["alliance_id"].unique()
    for alliance in alliances.loc[alliances['account_id']==account]['alliance_id'].unique():
        if alliance in private_chats:
            return 1
    return 0



accounts = messages_df.loc[messages_df["trigger"]==1]["account_id"]
for i in range(accounts.shape[0]):
    account = accounts.iloc[i]

    if number_kicks(account)>=1:
        print(i, number_kicks(account), check_time_kick(account))

    if small_clan(account) == 1:
        print("==============================================================" + str(i))

2 1 0.999697650462963
==============================================================9
==============================================================20
32 1 0.9804620833333333
==============================================================32
==============================================================33
==============================================================49
==============================================================51
71 1 0.999697650462963
==============================================================76
==============================================================91
97 1 0.999697650462963
==============================================================99
==============================================================108
==============================================================133
155 1 0.8929949537037037
==============================================================155
==============================================================175
===================================

In [ ]:
#ALEX function we don't choose
def check_feature_of_trigger_message(df, feature ):
	actual_triggers= df.loc[df['trigger'] == 1]
	mean_guy= actual_triggers.loc[:,'account_id'].drop_duplicates()
	if mean_guy.shape[0]==1:
		account_id_mean=mean_guy.iloc[0]
		score_mean_guys=(account_id_mean, df.loc[df['account_id']==account_id_mean][feature].astype(float).sum())

	else:
		account_id_mean=mean_guy.values.tolist()
		score_mean_guys=[]
		for i in mean_guy:
			score = df.loc[df['account_id']==i]['GENERAL_RISK'].astype(float).sum()
			print(score)
			score_mean_guys.append((i,score))
		

	return score_mean_guys



In [209]:
often_time_data = timedelta(hours=12)
def often_changed_alliance_fixed(account):
     return often_change_alliance(account,often_time_data)

fct_message = [
    (check_message_ask_age,0.3),
    (check_message_trigger,0.2),
    (check_message_ask_age,0. ),
    (check_message_trigger,0. ),
    (check_message_account,0. ),
    (check_message_compliment,0. ),
    (check_message_relationship,0. )
    
]
fct_account = [
    (nb_of_alliance_changes,0. ),
    (same_message_alliance,0. ),
    (often_changed_alliance_fixed,0. ),
    (check_time_kick,0. ),
    (number_kicks,0. ),
    (small_clan,0. ),
    (count_triggers_account,0. )
    
]




def compute_toke_charge(account, msg_list):
    token_charge = 0
    for msg in msg_list:
        for fct_msg,weight in fct_message:
            token_charge += weight * fct_msg(msg)
    for fct_acc,weight in fct_account:
        token_charge += weight * fct_acc(account)
    return token_charge 

account = 'c2110e17f133e01ce5b70e2fa1c5b846019ea36649897babf67dd69fc538d880'
msgs_of_account = messages_df[messages_df['account_id']==account]['raw_message']
compute_toke_charge(account,msgs_of_account)


his_alliances 10223    b6cfffaf068b1a0bfb75f7b5e5237607a8022ea86be5b4...
Name: alliance_id, dtype: object
his_alliances_id ['b6cfffaf068b1a0bfb75f7b5e5237607a8022ea86be5b48770e896aca05f0d6b']
his_alliances 10223    b6cfffaf068b1a0bfb75f7b5e5237607a8022ea86be5b4...
Name: alliance_id, dtype: object
nb_all: 1
his_alliances 10223    b6cfffaf068b1a0bfb75f7b5e5237607a8022ea86be5b4...
Name: alliance_id, dtype: object


0.2